# Simple examples of Pyspark ML

In [1]:
from pyspark.sql import SparkSession

# create a session
spark = SparkSession.builder.appName("Missing").getOrCreate()

In [2]:
# Read the dataset
training = spark.read.csv("test1.csv", header=True, inferSchema=True)
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [3]:
# check data types
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [4]:
#columns
training.columns

['Name', 'age', 'Experience', 'Salary']

We group Age and experience:
    
[Age, Experience] ------> new feature ------> independent feature

In [5]:
from pyspark.ml.feature import VectorAssembler

featureassembler = VectorAssembler(inputCols=["age", "Experience"],
                                  outputCol="Independent Feature")

In [6]:
# apply the transformation to "training"
output = featureassembler.transform(training)

In [7]:
output.show()

+---------+---+----------+------+-------------------+
|     Name|age|Experience|Salary|Independent Feature|
+---------+---+----------+------+-------------------+
|    Krish| 31|        10| 30000|        [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|         [30.0,8.0]|
|    Sunny| 29|         4| 20000|         [29.0,4.0]|
|     Paul| 24|         3| 20000|         [24.0,3.0]|
|   Harsha| 21|         1| 15000|         [21.0,1.0]|
|  Shubham| 23|         2| 18000|         [23.0,2.0]|
+---------+---+----------+------+-------------------+



Our input feature will be `Independent Feature`, and our output feature (target) will be `Salary`

In [8]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Feature']

We are interested in `Independent Feature` and `Salary` :

In [9]:
finalized_data = output.select(["Independent Feature", "Salary"])
finalized_data.show()

+-------------------+------+
|Independent Feature|Salary|
+-------------------+------+
|        [31.0,10.0]| 30000|
|         [30.0,8.0]| 25000|
|         [29.0,4.0]| 20000|
|         [24.0,3.0]| 20000|
|         [21.0,1.0]| 15000|
|         [23.0,2.0]| 18000|
+-------------------+------+



In [10]:
from pyspark.ml.regression import LinearRegression

# Split the data into train (75%) and test data (25%):
train_data, test_data = finalized_data.randomSplit([0.75, 0.25])
regressor = LinearRegression(featuresCol="Independent Feature",
                             labelCol= "Salary")

regressor = regressor.fit(train_data)

In [11]:
train_data.show()

+-------------------+------+
|Independent Feature|Salary|
+-------------------+------+
|         [23.0,2.0]| 18000|
|         [24.0,3.0]| 20000|
|         [29.0,4.0]| 20000|
|         [30.0,8.0]| 25000|
|        [31.0,10.0]| 30000|
+-------------------+------+



In [12]:
test_data.show()

+-------------------+------+
|Independent Feature|Salary|
+-------------------+------+
|         [21.0,1.0]| 15000|
+-------------------+------+



In [13]:
# coefficients 
regressor.coefficients

DenseVector([-323.2867, 1696.8066])

In [14]:
regressor.intercept

22295.299605312008

In [15]:
# Prediction
pred_results = regressor.evaluate(test_data)

In [16]:
pred_results.predictions.show()

+-------------------+------+------------------+
|Independent Feature|Salary|        prediction|
+-------------------+------+------------------+
|         [21.0,1.0]| 15000|17203.085755292603|
+-------------------+------+------------------+



In [17]:
pred_results.meanAbsoluteError, pred_results.meanSquaredError

(2203.0857552926027, 4853586.845173177)